# Momentum Screener


The gist of this methodology is essentially to:

    1.) Penalize for Volatility
    2.) Buy non-news driven momentum - FIP helps filter for this


Process:

1.) Omit top % decile of the most volatile stocks in our universe

2.) 1yr/6month Volatiltiy Adjusted Returns: Calculate 1yr and 6month volatiltiy-adjusted returns and sort them highest to lowest - based on a combined 1yr & 6month score

3.) FIP (Momentum Quality): Calculate FIP for the top 50% of the stocks from the vol-adjsuted reteurn screen

4.) Rank Tickers: Calculate combined score based on the 1yr/6month vol-adjusted score and also the FIP score

5.) Purchase the top 40-50 equities from the universe (or a % based)

6.) Volatility Targetting: Volatility Targetted position sizing based on desired portfolio vol

7.) Rebalance: Rebalance monthly OR rebalance based on EV > Cost of rebalancing

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

In [93]:
sp500_tickers = pd.read_csv('sp500_tickers.csv')  # Assuming you have a file of S&P500 tickers
sp500_tickers.head()

,ticker
0,NVDA
1,MSFT
2,AAPL
3,AMZN
4,META


# Omit Top Decile of Highest 1Y Volatility

In [100]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Historical Volatility'])

In [102]:
# Collect rows in a list for concatenation
rows_to_add = []

# Convert tickers column to a list
tickers = sp500_tickers['ticker'].tolist()

# Batch download all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:  # Iterate over the 'ticker' list
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()

        # Check if data is insufficient
        if ticker_data.empty or len(ticker_data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}") 

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Rolling window for std
        window = 252  

        # Calculate rolling std of daily returns
        ticker_data['Rolling_Std'] = ticker_data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        ticker_data['Rolling_Hist_Vol'] = ticker_data['Rolling_Std'] * np.sqrt(window)

        # Retrieve the last value of Vol Adjusted Return
        historical_vol = ticker_data['Rolling_Hist_Vol'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Historical Volatility': historical_vol})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

[*********************100%***********************]  893 of 893 completed

31 Failed downloads:
['NCR', 'JWN', 'PDCO', 'SWAV', 'AZPN', 'PACW', 'SYNH', 'OFC', 'LHCG', 'STOR', 'ETRN', 'SRC', 'UNVR', 'BRK.B', 'GPS', 'NARI', 'PNM', 'IAA', 'LSI', 'WWE', 'AIRC', 'UMPQ', 'RCM', 'ENV', 'SWN', 'TPX', 'NATI', 'NYCB', 'PDCE']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
['PVH']: Timeout('Failed to perform, curl: (28) Operation timed out after 10003 milliseconds with 10 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y)')
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\1680125838.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fi

In [104]:
df = df.sort_values('Historical Volatility', ascending=False) #highest Vol at the top
df.head()

,Ticker,Historical Volatility
541,WOLF,2.287804
265,SMCI,1.148221
720,RUN,1.105152
886,VSAT,1.052422
790,FL,1.003620


In [106]:
df = df.sort_values('Historical Volatility', ascending=False) #highest Vol at the top

# Determine the 90th percentile threshold (top 10% cutoff)
vol_threshold = df['Historical Volatility'].quantile(0.90)

# Drop tickers in the top decile
df = df[df['Historical Volatility'] <= vol_threshold]

df.head()

,Ticker,Historical Volatility
440,CNC,0.539264
472,ALGN,0.538459
397,TER,0.536193
890,OMCL,0.535949
628,X,0.534247


In [108]:
sp500_tickers = df # rename dataframe to sp500 tickers for the loops

sp500_tickers.drop(columns=['Historical Volatility'],inplace=True)
sp500_tickers.rename(columns={'Ticker' : 'ticker'},inplace=True)
sp500_tickers.head()

,ticker
440,CNC
472,ALGN
397,TER
890,OMCL
628,X


# Volatility Adjusted Returns

#### 1.) 1Y Volatility-Adjusted Returns

In [112]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [114]:
# Collect rows in a list for concatenation
rows_to_add = []

# Convert tickers column to a list
tickers = sp500_tickers['ticker'].tolist()

# Batch download 2 years of data for all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()

        # Check if data is insufficient
        if ticker_data.empty or len(ticker_data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        ticker_data['Percent Return'] = ((ticker_data['Close'] - ticker_data['Close'].shift(252)) / ticker_data['Close'].shift(252)) * 100  

        # Rolling window for std dev (daily returns)
        window = 252  

        # Calculate rolling standard deviation of daily returns
        ticker_data['Rolling_Std'] = ticker_data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        ticker_data['Rolling_Hist_Vol'] = ticker_data['Rolling_Std'] * np.sqrt(window)

        # Calculate volatility-adjusted returns
        ticker_data['Vol_Adjusted_Return'] = ((ticker_data['Percent Return'])/100) / (ticker_data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = ticker_data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

[*********************100%***********************]  775 of 775 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\2641018204.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\2641018204.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\2641018204.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a 

In [116]:
df = df.sort_values('Vol Adjusted Return', ascending=False)
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [118]:
# read the csv file - momentum screen results
df1 = pd.read_csv("Momentum Screen Results.csv")

df1['Index1'] = df1.index # create column of the index position
df1.head()

,Ticker,Vol Adjusted Return,Index1
0,TPR,4.498198,0
1,JBL,3.253323,1
2,DASH,3.245977,2
3,IBKR,3.088371,3
4,HWM,2.995266,4


### 2.) 6M Volatility Adjusted Returns

In [120]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [122]:
# Collect rows in a list for concatenation
rows_to_add = []

# Convert tickers column to a list
tickers = sp500_tickers['ticker'].tolist()

# Batch download 1 year of data for all tickers at once
data = yf.download(tickers, period='1y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()
        
        # Check if data is insufficient
        if ticker_data.empty or len(ticker_data) < 126:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        ticker_data['Percent Return'] = ((ticker_data['Close'] - ticker_data['Close'].shift(126)) / ticker_data['Close'].shift(126)) * 100  

        # Rolling window for std dev (daily returns)
        window = 126  

        # Calculate rolling standard deviation of daily returns
        ticker_data['Rolling_Std'] = ticker_data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        ticker_data['Rolling_Hist_Vol'] = ticker_data['Rolling_Std'] * np.sqrt(252)

        # Calculate volatility-adjusted returns
        ticker_data['Vol_Adjusted_Return'] = ((ticker_data['Percent Return'])/100) / (ticker_data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = ticker_data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

[*********************100%***********************]  775 of 775 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\3556766263.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\3556766263.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_11904\3556766263.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a 

In [126]:
df = df.sort_values('Vol Adjusted Return', ascending=False)
# Create a csv file with the results
df.to_csv("6 Month Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [128]:
df2 = pd.read_csv("6 Month Momentum Screen Results.csv")
df2['Index2'] = df2.index # create column of the index position
df2.head()

,Ticker,Vol Adjusted Return,Index2
0,NEU,1.719420,0
1,SXT,1.578279,1
2,APH,1.518146,2
3,LHX,1.410538,3
4,ORA,1.407010,4


### Merge Both Dataframes - 1Y and 6M vol volatility adjusted returns

In [130]:
# Perform an inner join to include only matching tickers
df = pd.merge(df1, df2, on='Ticker', how='inner')
df.head()

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2
0,TPR,4.498198,0,0.983924,19
1,JBL,3.253323,1,0.791529,51
2,DASH,3.245977,2,0.792263,50
3,IBKR,3.088371,3,0.180794,238
4,HWM,2.995266,4,0.999026,18


### Calculate Average Rank

In [132]:
# calculate average rank

df['average_rank'] = df['Index1'] + df['Index2'] # simply add the two ranks
df.head()

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank
0,TPR,4.498198,0,0.983924,19,19
1,JBL,3.253323,1,0.791529,51,52
2,DASH,3.245977,2,0.792263,50,52
3,IBKR,3.088371,3,0.180794,238,241
4,HWM,2.995266,4,0.999026,18,22


In [134]:
df = df.sort_values('average_rank', ascending=True)

df.reset_index(drop=True, inplace=True)

# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index
df.head()


###############

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank
0,APH,2.399607,13,1.518146,2,15
1,CNP,2.773074,6,1.180759,9,15
2,FHI,2.599810,9,1.185187,8,17
3,TPR,4.498198,0,0.983924,19,19
4,SXT,2.238709,19,1.578279,1,20


## Convexity of Returns

In [90]:
#only keep first 100 rows of the Momentum Screen Results 
df1 = df1.iloc[:100]

In [92]:
# Collect rows in a list for concatenation
rows_to_add_4 = []

# Initialize a results DataFrame
results_df = pd.DataFrame(columns=['Ticker', 'Convexity Ratio'])

for Ticker in df1['Ticker']:  # Iterate over the 'Ticker' column
    try:
        # Download 2 years of historical data
        data = yf.download(Ticker, period='2y')
        
        if data.empty or len(data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {Ticker}")
        
        # Create a DataFrame for returns
        df = pd.DataFrame()
        df['Return'] = data['Adj Close'].pct_change()

        # Calculate the slope using a rolling window
        window = 50  # Adjust window size as needed
        df['Slope'] = df['Return'].rolling(window).apply(
            lambda x: np.polyfit(np.arange(len(x)), x, 1)[0], raw=True
        )

        # Compute the second derivative of the slope
        df['Second_Derivative'] = df['Slope'].diff()

        # Calculate the convexity ratio
        df['Convexity_Ratio'] = df['Second_Derivative'] / df['Slope']
        
        # Extract the most recent convexity ratio
        convexity_ratio = df['Convexity_Ratio'].iloc[-1]
        
        # Add the result to the list
        rows_to_add_4.append({'Ticker': Ticker, 'Convexity Ratio': convexity_ratio})
    
    except Exception as e:
        print(f"Error retrieving data for {Ticker}: {str(e)}")
        
# Concatenate all rows into the DataFrame at once
results_df = pd.concat([pd.DataFrame(rows_to_add_4)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed


Error retrieving data for FOX: 'Adj Close'


[*********************100%***********************]  1 of 1 completed


Error retrieving data for FOXA: 'Adj Close'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Error retrieving data for COKE: 'Adj Close'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error retrieving data for PGR: 'Adj Close'
Error retrieving data for TTWO: 'Adj Close'


KeyboardInterrupt: 

In [56]:
df2 = results_df.sort_values(by='Convexity Ratio', ascending=False)

In [79]:
df2.head()

,Ticker,Convexity Ratio
46,MRK,7.440757
48,INTC,3.696518
44,PEP,2.642901
24,TXN,1.623321
20,XOM,1.375793


In [83]:
#Create a new csv file with our FIP results
df2.to_csv('Convexity_Results.csv', index=False)

# Convexity Based on Quadratic Regression

In [214]:
df = df.iloc[:400]

In [216]:
rows_to_add_2 = []

# Initialize an empty DataFrame to store results
df_convexity = pd.DataFrame(columns=['Ticker', 'Convexity Score'])

# Get tickers from df['Ticker'] list
tickers = df['Ticker'].tolist()

# Batch download 2 years of data for all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()

        # Keep only last 252 rows... 252 trading days
        ticker_data = ticker_data.tail(252)

        # Ensure there is sufficient data
        if ticker_data.empty or len(ticker_data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")
        
        # Calculate daily returns (percentage change)
        ticker_data['Return'] = ticker_data['Close'].pct_change() * 100

        # Drop the first row since the return for the first day is NaN
        ticker_data.dropna(inplace=True)
        
        # Create an ordinal time variable (index of the data)
        ticker_data.loc[:, 'Time'] = np.arange(len(ticker_data))
        
        # Prepare the independent variables (Time and Time^2 for quadratic regression)
        X = np.vstack([ticker_data['Time'], ticker_data['Time']**2]).T
        y = ticker_data['Return'].values
        
        # Perform a quadratic regression
        model = LinearRegression().fit(X, y)
        
        # The convexity is the coefficient of the squared time term
        convexity = model.coef_[1]  # This is the coefficient of Time^2 (quadratic term)
        
        # Add the result to the list
        rows_to_add_2.append({'Ticker': ticker, 'Convexity Score': convexity})
    
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df_convexity = pd.concat([pd.DataFrame(rows_to_add_2)], ignore_index=True)


[*********************100%***********************]  400 of 400 completed


In [284]:
df_convexity = df_convexity.sort_values('Convexity Score', ascending=False)
df_convexity.reset_index(drop=True, inplace=True)
df_convexity.head()

,Ticker,Convexity Score,Index3
0,LWAY,0.000160,0
1,RIGL,0.000158,1
2,XMTR,0.000148,2
3,UEC,0.000134,3
4,YOU,0.000128,4


In [224]:
df_convexity.to_csv("Convexity_Score_Quadratic_Method.csv")


###

# Momentum Quality - FIP (Frog in the Pan Score)

ID or FIP Score
ID = sign x [% Negative - % Positive]

In [136]:
# Only keep first 400 rows of the Momentum Screen Results
df = df.iloc[:400]

In [138]:
# Create a blank dataframe with the appropriate columns
df_FIP = pd.DataFrame(columns=['Ticker', 'FIP_Score'])

In [140]:
rows_to_add_3 = []

# Get list of tickers from df
tickers = df['Ticker'].tolist()

# Batch download 2 years of data for all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for Ticker in tickers:  # Iterate over the 'ticker' column in df
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[Ticker].copy()
        ticker_data = ticker_data.tail(252)  # Use only last 252 rows... for 252 trading days

        if ticker_data.empty or len(ticker_data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {Ticker}")
        
        ticker_data['Percent Return'] = (ticker_data['Close'].pct_change()) * 100

        if ticker_data['Percent Return'].cumsum().iat[-1] > 0:
            sign = 1
        elif ticker_data['Percent Return'].cumsum().iat[-1] < 0:
            sign = -1
        else:
            sign = 0

        positive_days = (ticker_data['Percent Return'] > 0).sum()
        negative_days = (ticker_data['Percent Return'] < 0).sum()
        flat_days = (ticker_data['Percent Return'] == 0).sum()
        pct_positive = round(positive_days / len(ticker_data), 4)
        pct_negative = round(negative_days / len(ticker_data), 4)
        FIP_Score = round(sign * (pct_negative - pct_positive), 4)

        # Add the result to the list
        rows_to_add_3.append({'Ticker': Ticker, 'FIP Score': FIP_Score})

    except Exception as e:
        print(f"Error retrieving data for {Ticker}: {str(e)}")

df_FIP = pd.concat([pd.DataFrame(rows_to_add_3)], ignore_index=True)


[*********************100%***********************]  391 of 391 completed


In [142]:
df_FIP = df_FIP.sort_values('FIP Score', ascending=True) # ascending = True since the best scores are negative
df_FIP.reset_index(drop=True, inplace=True)
df_FIP.head()

,Ticker,FIP Score
0,DTE,-0.2381
1,NDAQ,-0.2301
2,CW,-0.2222
3,GL,-0.2103
4,EVRG,-0.2024


In [144]:
# Create a new csv file with our FIP results
df_FIP.to_csv('Momentum_FIP_Results.csv', index=False)


###

# Combine Dataframes and Rank Tickers

In [146]:
#Create column of index #
df['index_vol_rank'] = df.index #vol adjusted returns df

#Create column of index #
df_FIP['index_fip_rank'] = df_FIP.index # FIP df

# create column of index # 
#df_convexity['Index3'] = df_convexity.index

In [148]:
# Perform an inner join to include only matching tickers
merged = pd.merge(df_FIP, df, on='Ticker', how='inner')

merged.head()

,Ticker,FIP Score,index_fip_rank,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank,index_vol_rank
0,DTE,-0.2381,0,1.038812,150,0.770732,55,205,69
1,NDAQ,-0.2301,1,2.172501,22,0.585190,92,114,30
2,CW,-0.2222,2,2.281018,17,0.973420,22,39,7
3,GL,-0.2103,3,2.128719,26,0.443613,130,156,52
4,EVRG,-0.2024,4,1.548051,67,0.690504,66,133,43


### Next cell is only necessary if adding a third filter to the ranking:

In [268]:
# Merge the result with the third DataFrame
merged = pd.merge(merged, df_convexity, on='Ticker', how='inner')

### Proceed:

In [150]:
# Calculate the score (sum of indices)

#merged['Score'] = merged['index_vol_rank'] + merged['index_fip_rank'] + merged['Index3']
merged['Score'] = merged['index_vol_rank'] + merged['index_fip_rank']

# Sort the result by the score
sorted_df = merged.sort_values('Score', ascending=True)
# Select relevant columns for the output
result = sorted_df[['Ticker', 'Score']]

In [152]:
result.head()

,Ticker,Score
2,CW,9
15,HWM,20
7,CME,20
9,DASH,20
13,CAH,21


In [71]:
result.to_csv("Momentum_Ranked.csv",index=False)






#####

# Risk-Parity Volatility Targetting

This methodology transforms a set of screened tickers into a risk‑controlled portfolio. By estimating each asset’s historical volatility and using inverse‑volatility weighting, the process allocates more capital to steadier names while reducing exposure to highly volatile ones. The final scaling step ensures that the entire portfolio is expected to experience a predefined annualised volatility (e.g. 20%), giving you a consistent risk profile regardless of market regime.

In [174]:
# keep only the top 50 equities
result = result.head(50)
result.head(50)

,Ticker,Score
2,CW,9
15,HWM,20
7,CME,20
9,DASH,20
13,CAH,21
6,PM,24
16,GE,25
10,BK,29
28,FHI,30
1,NDAQ,31


### Risk Parity Light*

In [176]:
# Create a blank DataFrame to store the results
df_voltarget = pd.DataFrame(columns=['Ticker', 'Volatility', 'Weight'])

In [178]:
# Collect rows in a list for concatenation
rows_to_add = []

# Extract tickers from the df
tickers = result['Ticker'].dropna().tolist()

# Download 2 years of data for all tickers
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

# Prepare a DataFrame to store daily returns
returns_df = pd.DataFrame()
blended_vol_dict = {}

# Loop through each ticker
for ticker in tickers:
    try:
        # Retrieve individual ticker data
        ticker_data = data[ticker].copy()

        # Ensure sufficient history
        if ticker_data.empty or len(ticker_data) < 252:
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Store daily returns
        returns_df[ticker] = ticker_data['Daily Return']

        # Calculate annualized realized volatilities
        vol_21 = ticker_data['Daily Return'].rolling(21).std().iloc[-1] * np.sqrt(252) # one month vol
        vol_126 = ticker_data['Daily Return'].rolling(63).std().iloc[-1] * np.sqrt(252) # 3 month vol
        #vol_252 = ticker_data['Daily Return'].rolling(126).std().iloc[-1] * np.sqrt(252) # 1 year vol

        # Equal-weighted blended vol
        blended_vol = np.mean([vol_21, vol_126])

        # Store in dict
        blended_vol_dict[ticker] = blended_vol

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Drop rows w missing values
returns_df = returns_df.dropna()

# Convert to Series
volatility = pd.Series(blended_vol_dict)

# Compute inverse-volatility weights (raw weights before scaling)
inverse_vol = 1 / volatility
raw_weights = inverse_vol / inverse_vol.sum()

# Compute annualized covariance matrix
cov_matrix = returns_df.cov() * 252

# Portfolio current volatility
current_vol = np.sqrt(raw_weights.T @ cov_matrix @ raw_weights)

# Target vol scaling
target_vol = 0.15
scaling_factor = target_vol / current_vol

# Final weights after scaling
final_weights = (raw_weights * scaling_factor) * 100

# Build final output rows
for ticker in final_weights.index:
    rows_to_add.append({
        'Ticker': ticker,
        'Volatility': volatility[ticker],
        'Weight': final_weights[ticker]
    })

# Final output DataFrame
df_voltarget = pd.DataFrame(rows_to_add)

df_voltarget.head(50)

[*********************100%***********************]  50 of 50 completed


,Ticker,Volatility,Weight
0,CW,0.320931,1.561526
1,HWM,0.302508,1.656626
2,CME,0.171384,2.924088
3,DASH,0.296250,1.691622
4,CAH,0.172242,2.909525
5,PM,0.315714,1.587330
6,GE,0.222672,2.250587
7,BK,0.160503,3.122317
8,FHI,0.186644,2.685015
9,NDAQ,0.211823,2.365849


### Full Risk Parity and Volatility Targetting (Accounts for Correlations)

In [190]:
# Create a blank DataFrame to store the results
df_voltarget = pd.DataFrame(columns=['Ticker', 'Volatility', 'Weight'])

In [206]:
# Risk parity solver
def solve_full_risk_parity(cov_matrix):
    n = cov_matrix.shape[0]
    init_w = np.ones(n) / n

    def portfolio_risk_contributions(weights):
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        marginal_contribs = cov_matrix @ weights
        risk_contribs = weights * marginal_contribs
        return risk_contribs / port_vol

    def objective(weights):
        rc = portfolio_risk_contributions(weights)
        return np.sum((rc - rc.mean())**2)

    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1) for _ in range(n)]

    result = minimize(objective, init_w, method='SLSQP', bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError("Risk parity optimization failed:", result.message)

    return pd.Series(result.x, index=cov_matrix.columns)

# Collect rows in a list for concatenation
rows_to_add = []

# Extract tickers from df
tickers = result['Ticker'].dropna().tolist()

# Download 2 years of data for all tickers
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

# Prepare a DataFrame to store daily returns
returns_df = pd.DataFrame()
blended_vol_dict = {}

# Loop through tickers
for ticker in tickers:
    try:
        ticker_data = data[ticker].copy()

        if ticker_data.empty or len(ticker_data) < 252:
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
        returns_df[ticker] = ticker_data['Daily Return']

        vol_21 = ticker_data['Daily Return'].rolling(21).std().iloc[-1] * np.sqrt(252) # one month vol
        vol_126 = ticker_data['Daily Return'].rolling(63).std().iloc[-1] * np.sqrt(252) # 3 month vol
        blended_vol = np.mean([vol_21, vol_126])
        blended_vol_dict[ticker] = blended_vol

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# drop rows w nan's
returns_df = returns_df.dropna()

# convert to volatility series
volatility = pd.Series(blended_vol_dict)

# Covariance matrix (annualized)
cov_matrix = returns_df.cov() * 252

# Risk Parity Weights
raw_weights = solve_full_risk_parity(cov_matrix)

# Portfolio vol (full risk parity weights)
current_vol = np.sqrt(raw_weights.T @ cov_matrix @ raw_weights)

# Vo targeting
target_vol = 0.15
scaling_factor = target_vol / current_vol
final_weights = (raw_weights * scaling_factor) * 100

# Output DataFrame
for ticker in final_weights.index:
    rows_to_add.append({
        'Ticker': ticker,
        'Volatility': volatility[ticker],
        'Weight': final_weights[ticker]
    })

df_voltarget = pd.DataFrame(rows_to_add)

df_voltarget.head(50)

[*********************100%***********************]  50 of 50 completed


,Ticker,Volatility,Weight
0,CW,0.313156,1.756360
1,HWM,0.302401,1.367595
2,CME,0.171463,3.340483
3,DASH,0.288751,1.416659
4,CAH,0.170746,3.246620
5,PM,0.316088,3.351167
6,GE,0.222464,1.527086
7,BK,0.160687,2.065763
8,FHI,0.186519,2.666943
9,NDAQ,0.211776,2.154816


In [208]:
print('Total Portfolio Exposure:')
print(df_voltarget['Weight'].sum().round(4))
print()
print('Average % Exposure:')
print(df_voltarget['Weight'].mean().round(4))

Total Portfolio Exposure:
117.4974

Average % Exposure:
2.3499


In [196]:
# save target portfolio into CSV file & txt. file

df_voltarget.to_csv('Target Portfolio.csv')

df_voltarget[['Ticker']].to_csv('Target_Portfolio.txt', sep='\t', index=False) # Export only the 'Ticker' column to a text file


######